# Patient Like Me - End to End

In [8]:
# !pip install "sagemaker>=2.140.0" "transformers==4.26.1" "datasets[s3]==2.10.1" --upgrade
# !pip install git+https://github.com/huggingface/transformers.git

# !pip install transformers datasets[s3] sagemaker --upgrade
# !pip install scikit-learn
# !pip install accelerate==0.20.3
# !pip install fastprogress
# !pip install ollama
# !ollama pull llama3
# !pip install llama-index==0.10.32
# !pip install langchain
# !pip install langchain_community

In [5]:
import numpy as np
from sagemaker import get_execution_role
import boto3
import pandas as pd
from io import StringIO # Python 3.
from datasets import load_dataset,Dataset,DatasetDict,concatenate_datasets
from fastprogress.fastprogress import master_bar, progress_bar

import pickle
from transformers import DataCollatorWithPadding,AutoModelForSequenceClassification, Trainer, TrainingArguments,AutoTokenizer,AutoModel,AutoConfig
from transformers.modeling_outputs import TokenClassifierOutput
import torch
import torch.nn as nn
import pandas as pd
import json

import matplotlib.pyplot as plt 
from sklearn.metrics import roc_curve, precision_recall_curve
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score
from transformers import TextClassificationPipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pickle
from transformers import AdamW, get_scheduler
from datasets import load_metric
from tqdm.auto import tqdm
from torch.utils.data import DataLoader
from sklearn.metrics import roc_auc_score, average_precision_score
import matplotlib.pyplot as plt 
from sklearn.metrics import roc_curve, precision_recall_curve
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score

import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from transformers import AutoTokenizer, AutoModel
import torch
from tqdm import tqdm
from collections import Counter

In [6]:
bucket_name = 'chianglab-dataderivatives'
file_path = "mimic-iv-ed-2.2/text_repr.json"

s3 = boto3.resource('s3')
content_object = s3.Object(bucket_name, file_path)
file_content = content_object.get()['Body'].read().decode('utf-8')
json_content = json.loads(file_content)
df = pd.DataFrame(json_content).T

df['eddischarge'] = [1 if 'admitted' in s.lower() else 0 for s in df['eddischarge']] # admitted = 1, Home = 0
df['medrecon'] = df['medrecon'].fillna("The patient was previously not taking any medications.")
df['pyxis'] = df['pyxis'].fillna("The patient did not receive any medications.")
df['vitals'] = df['vitals'].fillna("The patient had no vitals recorded")
df['codes'] = df['codes'].fillna("The patient received no diagnostic codes")
df = df.drop("admission",axis=1)
df = df.drop("discharge",axis=1)
df = df.drop("eddischarge_category",axis=1)
df['ID'] = df.arrival.astype(str).str.split().str[1].replace(",", " ", regex=True).to_list()
df["patient_info"] = df["arrival"] + " " + df["triage"] + " " + df["medrecon"] + " " + df["vitals"] + " " + df["codes"] + " " + df["pyxis"]
df = df[[col for col in df.columns if col != 'eddischarge'] + ['eddischarge']] # rearrange column to the end
df

,arrival,triage,medrecon,vitals,codes,pyxis,ID,patient_info,eddischarge
33258284,"Patient 10000032, a 52 year old white female, ...","At triage: temperature was 98.4, pulse was 70....",The patient was previously taking the followin...,The patient had the following vitals: At 2180-...,The patient received the following diagnostic ...,The patient did not receive any medications.,10000032,"Patient 10000032, a 52 year old white female, ...",1
38112554,"Patient 10000032, a 52 year old white female, ...","At triage: temperature was 98.9, pulse was 88....",The patient was previously taking the followin...,The patient had the following vitals: At 2180-...,The patient received the following diagnostic ...,The patient received the following medications...,10000032,"Patient 10000032, a 52 year old white female, ...",1
35968195,"Patient 10000032, a 52 year old white female, ...","At triage: temperature was 99.4, pulse was 105...",The patient was previously taking the followin...,The patient had the following vitals: At 2180-...,The patient received the following diagnostic ...,The patient received the following medications...,10000032,"Patient 10000032, a 52 year old white female, ...",1
32952584,"Patient 10000032, a 52 year old white female, ...","At triage: temperature was 97.8, pulse was 87....",The patient was previously taking the followin...,The patient had the following vitals: At 2180-...,The patient received the following diagnostic ...,The patient received the following medications...,10000032,"Patient 10000032, a 52 year old white female, ...",0
39399961,"Patient 10000032, a 52 year old white female, ...","At triage: temperature was 98.7, pulse was 77....",The patient was previously taking the followin...,The patient had the following vitals: At 2180-...,The patient received the following diagnostic ...,The patient received the following medications...,10000032,"Patient 10000032, a 52 year old white female, ...",1
...,...,...,...,...,...,...,...,...,...
34149746,"Patient 19999784, a 57 year old black/african ...","At triage: temperature was 98.8, pulse was 92....",The patient was previously taking the followin...,The patient had the following vitals: At 2119-...,The patient received the following diagnostic ...,The patient received the following medications...,19999784,"Patient 19999784, a 57 year old black/african ...",1
35692999,"Patient 19999784, a 57 year old black/african ...","At triage: temperature was 98.6, pulse was 80....",The patient was previously not taking any medi...,The patient had the following vitals: At 2119-...,The patient received the following diagnostic ...,The patient did not receive any medications.,19999784,"Patient 19999784, a 57 year old black/african ...",1
32917002,"Patient 19999828, a 46 year old white female, ...","At triage: temperature was 96.6, pulse was 112...",The patient was previously taking the followin...,The patient had the following vitals: At 2149-...,The patient received the following diagnostic ...,The patient received the following medications...,19999828,"Patient 19999828, a 46 year old white female, ...",1
30712109,"Patient 19999828, a 46 year old white female, ...","At triage: temperature was 98.1, pulse was 83....",The patient was previously taking the followin...,The patient had the following vitals: At 2147-...,The patient received the following diagnostic ...,The patient received the following medications...,19999828,"Patient 19999828, a 46 year old white female, ...",1


In [28]:
df = df.drop("arrival",axis=1)
df = df.drop("triage",axis=1)
df = df.drop("medrecon",axis=1)
df = df.drop("vitals",axis=1)
df = df.drop("codes",axis=1)
df = df.drop("pyxis",axis=1)

In [42]:
temp = df.sample(10000)

# KNN where we bootstrap a group of embeddings and get a more robust prediciton

In [45]:
def knn(query_text, data_source, model_name, n_neighbors=5, n_bootstrap=1000, sample_size=1000):
    # Bootstrap a dataframe
    df = pd.DataFrame(data_source)
    
    # Encode the text
    def encode_texts(model_name, texts):
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModel.from_pretrained(model_name)
        model.eval()
        if torch.cuda.is_available():
            model.to('cuda')
        embeddings = []
        with torch.no_grad():
            for text in tqdm(texts):
                encoded_input = tokenizer(text, return_tensors='pt', truncation=True, max_length=512, padding='max_length')
                if torch.cuda.is_available():
                    encoded_input = {key: val.to('cuda') for key, val in encoded_input.items()}
                output = model(**encoded_input)
                cls_embedding = output.last_hidden_state[:, 0, :]
                embeddings.append(cls_embedding.cpu().numpy())
        print("embeddings are generated")
        return np.vstack(embeddings)

    # Encode all text data
    all_embeddings = encode_texts(model_name, df['patient_info'].tolist())
    
    # Encode the query text
    query_embedding = encode_texts(model_name, [query_text])

    bootstrap_predictions = []
    
    for _ in tqdm(range(n_bootstrap), desc="Bootstrapping"):
        # Sample indices
        sample_indices = np.random.choice(len(df), size=sample_size, replace=True)
        
        # Get sample data
        sample_embeddings = all_embeddings[sample_indices]
        sample_df = df.iloc[sample_indices]

        # Build KNN embedding space
        knn = NearestNeighbors(n_neighbors=n_neighbors, metric='cosine')
        knn.fit(sample_embeddings)
        
        # Query KNN
        distances, indices = knn.kneighbors(query_embedding)
        
        # Get results
        results = sample_df.iloc[indices[0]]
        
        # Make prediction based on majority vote
        neighbor_labels = results['eddischarge'].tolist()
        prediction = Counter(neighbor_labels).most_common(1)[0][0]
        
        bootstrap_predictions.append(prediction)

    # Final prediction based on majority vote of all bootstrap iterations
    final_prediction = Counter(bootstrap_predictions).most_common(1)[0][0]
    
    # Calculate confidence
    confidence = Counter(bootstrap_predictions)[final_prediction] / n_bootstrap

    return final_prediction, confidence

In [46]:
query_text = df["patient_info"].iloc[0]
data_source = temp
model_name = "bert-base-uncased" 
prediction, confidence = knn(query_text, data_source, model_name)
print(f"Predicted eddischarge: {prediction}")
print(f"Ground Truth: {df['eddischarge'].iloc[0]}")
print(f"Confidence: {confidence:.2f}")

100%|██████████| 10000/10000 [02:05<00:00, 79.70it/s]


embeddings are generated


100%|██████████| 1/1 [00:00<00:00, 77.52it/s]


embeddings are generated


Bootstrapping: 100%|██████████| 1000/1000 [00:03<00:00, 271.03it/s]

Predicted eddischarge: 0
Ground Truth: 1
Confidence: 0.78


In [47]:
query_text = df["patient_info"].iloc[0]
data_source = temp
model_name = "medicalai/ClinicalBERT"
prediction, confidence = knn(query_text, data_source, model_name)
print(f"Predicted eddischarge: {prediction}")
print(f"Ground Truth: {df['eddischarge'].iloc[0]}")
print(f"Confidence: {confidence:.2f}")

100%|██████████| 10000/10000 [01:11<00:00, 140.37it/s]


embeddings are generated


100%|██████████| 1/1 [00:00<00:00, 132.01it/s]


embeddings are generated


Bootstrapping: 100%|██████████| 1000/1000 [00:03<00:00, 277.14it/s]

Predicted eddischarge: 1
Ground Truth: 1
Confidence: 1.00


# KNN where we return the results table

In [44]:
def query_knn_embeddings(query_text, data_source, model_name, n_neighbors=5):
    # Bootstrap a dataframe
    df = pd.DataFrame(data_source)

    # Encode the text
    def encode_texts(model_name, texts):
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModel.from_pretrained(model_name)
        model.eval()
        if torch.cuda.is_available():
            model.to('cuda')
        embeddings = []
        with torch.no_grad():
            for text in tqdm(texts):
                encoded_input = tokenizer(text, return_tensors='pt', truncation=True, max_length=512, padding='max_length')
                if torch.cuda.is_available():
                    encoded_input = {key: val.to('cuda') for key, val in encoded_input.items()}
                output = model(**encoded_input)
                cls_embedding = output.last_hidden_state[:, 0, :]
                embeddings.append(cls_embedding.cpu().numpy())
        print("embeddings are generated")
        return np.vstack(embeddings)
    # Assume the text column is named 'patient_info'. Adjust if necessary.
    text_embeddings = encode_texts(model_name, df['patient_info'].tolist())

    # Build KNN embedding space
    knn = NearestNeighbors(n_neighbors=n_neighbors, metric='cosine')
    knn.fit(text_embeddings)

    # Encode the query text
    query_embedding = encode_texts(model_name, [query_text])

    # Query KNN
    distances, indices = knn.kneighbors(query_embedding)

    # Get results
    results = df.iloc[indices[0]]
    results['distance'] = distances[0]

    # Make prediction based on majority vote
    neighbor_labels = results['eddischarge'].tolist()
    prediction = Counter(neighbor_labels).most_common(1)[0][0]

    return results, prediction

In [38]:
# Example
query_text = df["patient_info"].iloc[0]
data_source = temp
model_name = "bert-base-uncased" # or any other model name
results, prediction = query_knn_embeddings(query_text, data_source, model_name)
print(results)
print("-------")
print(f"Predicted eddischarge: {prediction}")


100%|██████████| 1000/1000 [00:12<00:00, 80.19it/s]


embeddings are generated


100%|██████████| 1/1 [00:00<00:00, 77.90it/s]

embeddings are generated
                 ID                                       patient_info  \
32394613  18125751   Patient 18125751, a 57 year old white female, ...   
34022538  14187451   Patient 14187451, a 53 year old black/african ...   
35671330  18242530   Patient 18242530, a 76 year old white female, ...   
37661549  14062869   Patient 14062869, a 41 year old white female, ...   
32129835  12019283   Patient 12019283, a 25 year old black/african ...   

          eddischarge  distance  
32394613            0  0.013725  
34022538            1  0.016040  
35671330            0  0.016175  
37661549            0  0.016219  
32129835            0  0.017117  
-------
Predicted eddischarge: 0
Ground Truth: 1



<ipython-input-38-e356481266bc>:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results['distance'] = distances[0]


In [39]:
query_text = df["patient_info"].iloc[0]
data_source = temp
model_name = "medicalai/ClinicalBERT" 
results, prediction = query_knn_embeddings(query_text, data_source, model_name)
print(results)
print("-------")
print(f"Predicted eddischarge: {prediction}")
print(f"Ground Truth: {df['eddischarge'].iloc[0]}")

100%|██████████| 1000/1000 [00:07<00:00, 141.10it/s]


embeddings are generated


100%|██████████| 1/1 [00:00<00:00, 134.61it/s]

embeddings are generated
                 ID                                       patient_info  \
36599058  16315929   Patient 16315929, a 62 year old white female, ...   
32576195  13374041   Patient 13374041, a 58 year old white female, ...   
33509281  10018862   Patient 10018862, a 56 year old white female, ...   
31725842  18307993   Patient 18307993, a 45 year old black/african ...   
32638903  13471464   Patient 13471464, a 73 year old white female, ...   

          eddischarge  distance  
36599058            1  0.000650  
32576195            1  0.000689  
33509281            1  0.000711  
31725842            1  0.000743  
32638903            1  0.000770  
-------
Predicted eddischarge: 1
Ground Truth: 1



<ipython-input-38-e356481266bc>:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results['distance'] = distances[0]


# TODO: We can make plotting features and train test split next

